# Tutorial 12: Advanced LangChain Techniques

In this tutorial, we'll explore advanced LangChain techniques, including custom chain development, advanced prompt engineering, retrieval-augmented generation (RAG), and fine-tuning language models for specific tasks.

## Setup

First, let's import the necessary libraries and set up our environment:

In [9]:
import os
from typing import List, Dict, Any
from langchain_groq import ChatGroq
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.chains import LLMChain, SequentialChain, TransformChain
from langchain.chains.base import Chain
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_ollama import OllamaEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from pydantic import BaseModel, Field

# Initialize Groq LLM
llm = ChatGroq(
        model_name="llama-3.1-70b-versatile",
        temperature=0.7,
        model_kwargs={"top_p": 0.8, "seed": 1337}
    )

## 1. Custom chain development

Let's create a custom chain that processes text through multiple steps:

In [10]:
class TextProcessingChain(Chain):
    input_key: str = "input_text"
    output_key: str = "processed_text"

    @property
    def input_keys(self) -> List[str]:
        return [self.input_key]

    @property
    def output_keys(self) -> List[str]:
        return [self.output_key]

    def _call(self, inputs: Dict[str, str]) -> Dict[str, str]:
        text = inputs[self.input_key]
        
        # Step 1: Convert to lowercase
        text = text.lower()
        
        # Step 2: Remove punctuation
        text = ''.join(char for char in text if char.isalnum() or char.isspace())
        
        # Step 3: Remove extra whitespace
        text = ' '.join(text.split())
        
        return {self.output_key: text}

# Use the custom chain
text_processor = TextProcessingChain()
result = text_processor.run("Hello, World! This is a   TEST.")
print(result)

hello world this is a test


## 2. Prompt templating and management

Let's explore advanced prompt templating techniques:

In [11]:
# Define a complex prompt template
complex_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant specializing in {domain}."),
    ("human", "I need help with the following task: {task}"),
    ("ai", "Certainly! I'd be happy to help you with that. Could you provide more details about {detail_request}?"),
    ("human", "Here's more information: {additional_info}"),
    ("ai", "Thank you for the additional information. Based on what you've told me, here's my response:"),
])

# Create a chain using the complex prompt
complex_chain = LLMChain(llm=llm, prompt=complex_prompt)

# Run the chain
result = complex_chain.run({
    "domain": "software development",
    "task": "optimizing a slow database query",
    "detail_request": "the current query structure and database schema",
    "additional_info": "The query is a JOIN operation across three tables with multiple WHERE clauses. The database is PostgreSQL."
})

print(result)

 

**Optimizing a Slow JOIN Query in PostgreSQL**

To optimize a slow JOIN query in PostgreSQL, we can follow these steps:

### 1. **Analyze the Query Plan**

First, we need to analyze the query plan to identify the bottlenecks. You can use the `EXPLAIN` statement to get the query plan.

```sql
EXPLAIN (ANALYZE) SELECT * FROM table1
JOIN table2 ON table1.id = table2.id
JOIN table3 ON table2.id = table3.id
WHERE table1.column1 = 'value1'
AND table2.column2 = 'value2'
AND table3.column3 = 'value3';
```

This will give you the actual query plan, including the execution time and the number of rows processed.

### 2. **Indexing**

Make sure that the columns used in the `JOIN` and `WHERE` clauses are indexed. Indexing can significantly speed up the query.

```sql
CREATE INDEX idx_table1_column1 ON table1 (column1);
CREATE INDEX idx_table2_column2 ON table2 (column2);
CREATE INDEX idx_table3_column3 ON table3 (column3);
```

### 3. **Reorder the JOINs**

The order of the `JOIN`s can affect th

## 3. Implementing retrieval-augmented generation (RAG)

Let's implement a simple RAG system using FAISS and HuggingFace embeddings:

In [13]:
# Load and preprocess the document
loader = TextLoader("sample_document.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

# Create embeddings and vector store
embeddings = OllamaEmbeddings(model="all-minilm",base_url=os.getenv('OLLAMA_EMBEDDING_URL'))
db = FAISS.from_documents(texts, embeddings)

# Define the RAG prompt
rag_prompt = PromptTemplate(
    template="""Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Answer: """,
    input_variables=["context", "question"]
)

def retrieve_and_generate(query: str) -> str:
    # Retrieve relevant documents
    docs = db.similarity_search(query)
    context = "\n".join([doc.page_content for doc in docs])
    
    # Generate answer
    chain = LLMChain(llm=llm, prompt=rag_prompt)
    return chain.run({"context": context, "question": query})

# Test the RAG system
result = retrieve_and_generate("What is the capital of France?")
print(result)

The capital of France is Paris.


## 4. Fine-tuning language models for specific tasks

While we can't directly fine-tune the Groq model, we can simulate fine-tuning by creating a specialized prompt that adapts the model's behavior for a specific task. Let's create a 'fine-tuned' model for sentiment analysis:

In [14]:
sentiment_prompt = PromptTemplate(
    template="""You are a sentiment analysis expert. Your task is to analyze the sentiment of the given text and classify it as positive, negative, or neutral. 
    Provide a brief explanation for your classification.

    Text: {input_text}

    Sentiment: """,
    input_variables=["input_text"]
)

sentiment_chain = LLMChain(llm=llm, prompt=sentiment_prompt)

def analyze_sentiment(text: str) -> str:
    return sentiment_chain.run(text)

# Test the 'fine-tuned' sentiment analysis model
texts = [
    "I love this product! It's amazing and works perfectly.",
    "This is the worst experience I've ever had. Terrible customer service.",
    "The weather is quite nice today, not too hot or cold."
]

for text in texts:
    print(f"Text: {text}")
    print(f"Sentiment: {analyze_sentiment(text)}\n")

Text: I love this product! It's amazing and works perfectly.
Sentiment: Sentiment: Positive

Explanation: The text expresses a strong positive sentiment, as indicated by the use of enthusiastic language such as "I love", "amazing", and "works perfectly". The tone is energetic and affirmative, conveying a high level of satisfaction with the product.

Text: This is the worst experience I've ever had. Terrible customer service.
Sentiment: Sentiment: Negative

Explanation: The text expresses strong dissatisfaction with the experience and specifically mentions "terrible customer service". The use of words like "worst" and "terrible" convey a strong negative sentiment, indicating that the speaker is extremely unhappy with the service they received.

Text: The weather is quite nice today, not too hot or cold.
Sentiment: Sentiment: Neutral

Explanation: The text describes the weather as "quite nice," which has a slightly positive connotation. However, it is also described in a relatively neutr

## Conclusion

In this tutorial, we've explored advanced LangChain techniques, including:

1. Custom chain development for specialized text processing
2. Advanced prompt templating and management for complex interactions
3. Implementing a retrieval-augmented generation (RAG) system
4. Simulating fine-tuning for specific tasks using specialized prompts

These techniques allow you to create more sophisticated and tailored AI applications using LangChain and LangGraph.

## Next Steps

To further advance your skills with LangChain and LangGraph, consider:

1. Experimenting with more complex custom chains and combining them with LangGraph flows
2. Developing a prompt management system for large-scale applications
3. Exploring advanced RAG techniques, such as hypothetical document embeddings or multi-query retrieval
4. Investigating ways to evaluate and improve the performance of your 'fine-tuned' models
5. Integrating these advanced techniques into a full-fledged application